In [1]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 19.6 MB/s eta 0:00:00a 0:00:01


In [2]:
#create samples :
import os
import torch
import pickle
import pandas as pd

def load_graph(path, is_pickle=True):
    """
    Load a molecule graph (.pkl) or a protein graph (.pt).
    If is_pickle is True, use pickle to load the file; otherwise, use torch.load.
    """
    if is_pickle:
        with open(path, 'rb') as f:
            return pickle.load(f)
    else:
        return torch.load(path)

def prepare_dataset_individual_save_as_pt(filtered_dataset, molecule_graph_dir, protein_graph_dir, output_dir):
    """
    Incrementally prepares the dataset and saves each (molecule, protein, target) tuple as a separate .pt file.

    Args:
    - filtered_dataset: The filtered KIBA dataset (DataFrame).
    - molecule_graph_dir: Directory where molecule graphs are stored.
    - protein_graph_dir: Directory where protein graphs are stored.
    - output_dir: Directory to save the prepared dataset incrementally.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for index, row in filtered_dataset.iterrows():
        protein_id = row['Target_ID']
        chembl_id = row['Drug_ID']

        # Load the protein graph (.pt)
        pro_graph_path = os.path.join(protein_graph_dir, f"{protein_id}_graph.pt")
        if not os.path.exists(pro_graph_path):
            print(f"Protein graph not found: {protein_id}")
            continue
        pro_graph = load_graph(pro_graph_path, is_pickle=False)

        # Load the molecule graph (.pkl)
        mol_graph_path = os.path.join(molecule_graph_dir, f"{chembl_id}_graph.pkl")
        if not os.path.exists(mol_graph_path):
            print(f"Molecule graph not found: {chembl_id}")
            continue
        mol_graph = load_graph(mol_graph_path)

        # Load target (affinity value)
        target = torch.tensor([row['Y']], dtype=torch.float)

        # Create the sample as a tuple (molecule graph, protein graph, target)
        sample = (mol_graph, pro_graph, target)

        # Save the sample as a .pt file
        sample_path = os.path.join(output_dir, f"sample_{index}.pt")
        torch.save(sample, sample_path)

        if(index%10000 == 0 ):
            print(f"Saved sample {index} as {sample_path}")




# Example usage for individual saving
molecule_graph_dir = '/kaggle/input/graphs/molecule_graphs'  # Directory where molecule graphs are stored
protein_graph_dir = '/kaggle/input/graphs/ProteinGraphs'  # Directory where protein graphs are stored
filtered_dataset_path = '/kaggle/input/graphs/filtered_DavisDataSet.csv'  # Path to the filtered dataset CSV
output_dir = 'prepared_samples/'  # Directory to save individual samples

# Load filtered dataset CSV
filtered_dataset = pd.read_csv(filtered_dataset_path)

# Prepare the dataset incrementally, saving each sample as a .pt file
prepare_dataset_individual_save_as_pt(filtered_dataset, molecule_graph_dir, protein_graph_dir, output_dir)

print("Dataset preparation completed.")


<ipython-input-2-c90b9d14e3c5>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path)


Saved sample 0 as prepared_samples/sample_0.pt


<ipython-input-2-c90b9d14e3c5>:16: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(path)


Saved sample 10000 as prepared_samples/sample_10000.pt
Dataset preparation completed.


In [ ]:
#model 1
import os
import warnings
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.nn import MSELoss
from torch_geometric.nn import GCNConv, global_mean_pool as gep
from torch_geometric.data import Data, Batch
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np

# Optional, for plotting:
import matplotlib.pyplot as plt

##############################################################################
#                               1. METRICS
##############################################################################

@torch.no_grad()
def ci_vectorized(preds: torch.Tensor, targets: torch.Tensor) -> float:
    """
    Naive O(N^2) Concordance Index using vectorized GPU operations.
    preds, targets: (N,) on the same device (e.g. cuda).
    """
    # (N, N) differences
    p_diff = preds.unsqueeze(1) - preds.unsqueeze(0)
    t_diff = targets.unsqueeze(1) - targets.unsqueeze(0)
    # mask out pairs where targets are identical
    mask = (t_diff != 0)
    # sign of product => +1 (concordant), 0 (tie), -1 (discordant)
    sign_mat = torch.sign(p_diff * t_diff)
    # step function h(x): 1 if x>0, 0.5 if x=0, 0 if x<0
    h = (sign_mat == 1).float() + 0.5 * (sign_mat == 0).float()
    # apply mask
    h_masked = h * mask.float()
    c = h_masked.sum()
    s = mask.sum().float()
    return (c / s).item() if s > 0 else 0.0

@torch.no_grad()
def mse_torch(preds: torch.Tensor, targets: torch.Tensor) -> float:
    """
    GPU-friendly MSE using torch's built-in mean squared error.
    """
    return F.mse_loss(preds, targets, reduction='mean').item()

@torch.no_grad()
def pearson_torch(preds: torch.Tensor, targets: torch.Tensor) -> float:
    """
    Pearson correlation coefficient in PyTorch (GPU-friendly).
    preds, targets: (N,) on the same device
    """
    p_centered = preds - preds.mean()
    t_centered = targets - targets.mean()
    cov = (p_centered * t_centered).sum()
    denom = torch.sqrt((p_centered**2).sum()) * torch.sqrt((t_centered**2).sum())
    eps = 1e-8
    return (cov / (denom + eps)).item()

##############################################################################
#                       2. GNN MODEL DEFINITION
##############################################################################

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, global_mean_pool as gep

#############################################
# Bidirectional Cross-Attention Module
#############################################
# class BidirectionalCrossAttention(nn.Module):
#     def __init__(self, embed_dim=128, num_heads=4):
#         super().__init__()
#         """
#         Bidirectional Cross-Attention Formulas:
        
#         For Graph A → Graph B:
#             Q_A = X_A W_Q_A, K_B = X_B W_K_A, V_B = X_B W_V_A
#             Z_A = softmax(Q_A K_B^T / √d) V_B
            
#         For Graph B → Graph A:
#             Q_B = X_B W_Q_B, K_A = X_A W_K_B, V_A = X_A W_V_B
#             Z_B = softmax(Q_B K_A^T / √d) V_A
#         """
#         self.embed_dim = embed_dim
        
#         # Graph A → Graph B weights
#         self.W_Q_A = nn.Linear(embed_dim, embed_dim)
#         self.W_K_A = nn.Linear(embed_dim, embed_dim)
#         self.W_V_A = nn.Linear(embed_dim, embed_dim)
        
#         # Graph B → Graph A weights
#         self.W_Q_B = nn.Linear(embed_dim, embed_dim)
#         self.W_K_B = nn.Linear(embed_dim, embed_dim)
#         self.W_V_B = nn.Linear(embed_dim, embed_dim)
        
#         self.scale = 1.0 / torch.sqrt(torch.tensor(embed_dim, dtype=torch.float32))

#     def forward(self, X_A, X_B, batch_A, batch_B):
#         """
#         Args:
#             X_A: Graph A node embeddings [total_A_nodes, embed_dim]
#             X_B: Graph B node embeddings [total_B_nodes, embed_dim]
#             batch_A: Batch indices for Graph A [total_A_nodes]
#             batch_B: Batch indices for Graph B [total_B_nodes]
#         Returns:
#             Z_A: Updated embeddings for Graph A (same shape as X_A)
#             Z_B: Updated embeddings for Graph B (same shape as X_B)
#         """
#         # --- Graph A updates from Graph B ---
#         Q_A = self.W_Q_A(X_A)  # [N_A, embed_dim]
#         K_B = self.W_K_A(X_B)  # [N_B, embed_dim]
#         V_B = self.W_V_A(X_B)  # [N_B, embed_dim]
        
#         scores_AB = (Q_A @ K_B.T) * self.scale  # [N_A, N_B]
#         mask_AB = self._create_mask(batch_A, batch_B)  # [N_A, N_B]
#         scores_AB = scores_AB.masked_fill(~mask_AB, -1e9)
#         attn_AB = F.softmax(scores_AB, dim=-1)
#         Z_A = attn_AB @ V_B  # Updated Graph A features
        
#         # --- Graph B updates from Graph A ---
#         Q_B = self.W_Q_B(X_B)  # [N_B, embed_dim]
#         K_A = self.W_K_B(X_A)  # [N_A, embed_dim]
#         V_A = self.W_V_B(X_A)  # [N_A, embed_dim]
        
#         scores_BA = (Q_B @ K_A.T) * self.scale  # [N_B, N_A]
#         mask_BA = mask_AB.T  # [N_B, N_A]
#         scores_BA = scores_BA.masked_fill(~mask_BA, -1e9)
#         attn_BA = F.softmax(scores_BA, dim=-1)
#         Z_B = attn_BA @ V_A  # Updated Graph B features
        
#         return Z_A, Z_B

#     def _create_mask(self, batch_A, batch_B):
#         # Creates a mask [N_A, N_B] so that nodes only attend to nodes from the same sample.
#         return batch_A.unsqueeze(1) == batch_B.unsqueeze(0)

import torch
import torch.nn as nn
import torch.nn.functional as F

class BidirectionalCrossAttention(nn.Module):
    def __init__(self, embed_dim=128, num_heads=4, chunk_size=2048):
        super().__init__()
        """
        Bidirectional Cross-Attention Formulas:
        
        For Graph A → Graph B:
            Q_A = X_A W_Q_A, K_B = X_B W_K_A, V_B = X_B W_V_A
            Z_A = softmax(Q_A K_B^T / √d) V_B
            
        For Graph B → Graph A:
            Q_B = X_B W_Q_B, K_A = X_A W_K_B, V_A = X_A W_V_B
            Z_B = softmax(Q_B K_A^T / √d) V_A
        """
        self.embed_dim = embed_dim
        self.chunk_size = chunk_size  # Adjust this value to control memory usage
        
        # Graph A → Graph B weights
        self.W_Q_A = nn.Linear(embed_dim, embed_dim)
        self.W_K_A = nn.Linear(embed_dim, embed_dim)
        self.W_V_A = nn.Linear(embed_dim, embed_dim)
        
        # Graph B → Graph A weights
        self.W_Q_B = nn.Linear(embed_dim, embed_dim)
        self.W_K_B = nn.Linear(embed_dim, embed_dim)
        self.W_V_B = nn.Linear(embed_dim, embed_dim)
        
        self.scale = 1.0 / torch.sqrt(torch.tensor(embed_dim, dtype=torch.float32))

    def forward(self, X_A, X_B, batch_A, batch_B):
        """
        Args:
            X_A: Graph A node embeddings [total_A_nodes, embed_dim]
            X_B: Graph B node embeddings [total_B_nodes, embed_dim]
            batch_A: Batch indices for Graph A [total_A_nodes]
            batch_B: Batch indices for Graph B [total_B_nodes]
        Returns:
            Z_A: Updated embeddings for Graph A (same shape as X_A)
            Z_B: Updated embeddings for Graph B (same shape as X_B)
        """
        # Compute linear projections for both directions
        Q_A = self.W_Q_A(X_A)  # [N_A, embed_dim]
        K_B = self.W_K_A(X_B)  # [N_B, embed_dim]
        V_B = self.W_V_A(X_B)  # [N_B, embed_dim]
        
        Q_B = self.W_Q_B(X_B)  # [N_B, embed_dim]
        K_A = self.W_K_B(X_A)  # [N_A, embed_dim]
        V_A = self.W_V_B(X_A)  # [N_A, embed_dim]
        
        # Create mask so that only nodes from the same sample attend to each other
        mask_AB = self._create_mask(batch_A, batch_B)  # [N_A, N_B]
        mask_BA = mask_AB.T  # [N_B, N_A]
        
        # Compute Z_A in chunks to reduce memory usage
        Z_A_chunks = []
        for i in range(0, Q_A.size(0), self.chunk_size):
            Q_A_chunk = Q_A[i:i+self.chunk_size]  # [chunk_size, embed_dim]
            # Compute attention scores for this chunk: [chunk_size, N_B]
            scores_chunk = (Q_A_chunk @ K_B.T) * self.scale
            mask_chunk = mask_AB[i:i+self.chunk_size]  # [chunk_size, N_B]
            scores_chunk = scores_chunk.masked_fill(~mask_chunk, -1e9)
            attn_chunk = F.softmax(scores_chunk, dim=-1)
            Z_A_chunk = attn_chunk @ V_B  # [chunk_size, embed_dim]
            Z_A_chunks.append(Z_A_chunk)
        Z_A = torch.cat(Z_A_chunks, dim=0)
        
        # Similarly, compute Z_B in chunks along the query dimension of Graph B
        Z_B_chunks = []
        for i in range(0, Q_B.size(0), self.chunk_size):
            Q_B_chunk = Q_B[i:i+self.chunk_size]  # [chunk_size, embed_dim]
            scores_chunk = (Q_B_chunk @ K_A.T) * self.scale  # [chunk_size, N_A]
            mask_chunk = mask_BA[i:i+self.chunk_size]  # [chunk_size, N_A]
            scores_chunk = scores_chunk.masked_fill(~mask_chunk, -1e9)
            attn_chunk = F.softmax(scores_chunk, dim=-1)
            Z_B_chunk = attn_chunk @ V_A  # [chunk_size, embed_dim]
            Z_B_chunks.append(Z_B_chunk)
        Z_B = torch.cat(Z_B_chunks, dim=0)
        
        return Z_A, Z_B

    def _create_mask(self, batch_A, batch_B):
        """Creates a mask [N_A, N_B] so that nodes only attend to nodes from the same sample."""
        return batch_A.unsqueeze(1) == batch_B.unsqueeze(0)


#############################################
# Updated GNN Model with Cross-Attention (No Extra Projection)
#############################################
class GNNNet(torch.nn.Module):
    def __init__(self, n_output=1, num_features_pro=54, num_features_mol=78, output_dim=128, dropout=0.2):
        super(GNNNet, self).__init__()
        print('GNNNet Loaded')
        self.n_output = n_output
        
        # --- Molecule branch GCN layers ---
        self.mol_conv1 = GCNConv(num_features_mol, num_features_mol)
        # Set GCN2 to output 'output_dim' so that both branches share the same dimension.
        self.mol_conv2 = GCNConv(num_features_mol, 156)
        # GCN3 will process the updated features; input dim = output_dim.
        self.mol_conv3 = GCNConv(156, num_features_mol*4)
        
        self.mol_fc_g1 = nn.Linear(num_features_mol*4, 1024)
        self.mol_fc_g2 = nn.Linear(1024, output_dim)
        
        # --- Protein branch GCN layers ---
        self.pro_conv1 = GCNConv(num_features_pro, num_features_pro)
        self.pro_conv2 = GCNConv(num_features_pro, 156)
        self.pro_conv3 = GCNConv(156, num_features_pro*4)
        
        self.pro_fc_g1 = nn.Linear(num_features_pro*4, 1024)
        self.pro_fc_g2 = nn.Linear(1024, output_dim)
        
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
        # --- Bidirectional Cross-Attention ---
        # Now the node features from both branches are already in R^(output_dim)
        self.cross_attn = BidirectionalCrossAttention(embed_dim=156, num_heads=4)
        
        # --- Combined FC layers ---
        self.fc1 = nn.Linear(2 * output_dim, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.out = nn.Linear(512, self.n_output)

    def forward(self, data_mol, data_pro):
        # Unpack molecule graph data
        mol_x, mol_edge_index, mol_batch = data_mol.x, data_mol.edge_index, data_mol.batch
        # Unpack protein graph data
        target_x, target_edge_index, target_batch = data_pro.x, data_pro.edge_index, data_pro.batch

        # --- Molecule branch: GCN1 -> GCN2 ---
        x = self.mol_conv1(mol_x, mol_edge_index)
        x = self.relu(x)
        x = self.mol_conv2(x, mol_edge_index)
        x = self.relu(x)
        # x now has shape: [total_mol_nodes, output_dim]
        
        # --- Protein branch: GCN1 -> GCN2 ---
        xt = self.pro_conv1(target_x, target_edge_index)
        xt = self.relu(xt)
        xt = self.pro_conv2(xt, target_edge_index)
        xt = self.relu(xt)
        # xt now has shape: [total_pro_nodes, output_dim]
        
        # --- Apply bidirectional cross-attention ---
        # The cross-attention module uses batch indices to ensure nodes only attend to nodes from the same sample.
        Xnew, XtNew = self.cross_attn(x, xt, mol_batch, target_batch)
        
        # --- Continue with third GCN layer using the updated features ---
        x_updated = self.mol_conv3(Xnew, mol_edge_index)
        x_updated = self.relu(x_updated)
        x_pooled = gep(x_updated, mol_batch)  # Global pooling for molecule graphs
        
        xt_updated = self.pro_conv3(XtNew, target_edge_index)
        xt_updated = self.relu(xt_updated)
        xt_pooled = gep(xt_updated, target_batch)  # Global pooling for protein graphs
        
        # --- Flatten branch outputs through FC layers ---
        x_final = self.mol_fc_g2(self.relu(self.mol_fc_g1(x_pooled)))
        x_final = self.dropout(x_final)
        xt_final = self.pro_fc_g2(self.relu(self.pro_fc_g1(xt_pooled)))
        xt_final = self.dropout(xt_final)
        
        # --- Concatenate and apply combined FC layers ---
        xc = torch.cat((x_final, xt_final), dim=1)
        xc = self.fc1(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        xc = self.fc2(xc)
        xc = self.relu(xc)
        xc = self.dropout(xc)
        out = self.out(xc)
        
        return out

##############################################################################
#                   3. DATA LOADING HELPERS
##############################################################################

warnings.filterwarnings('ignore', category=FutureWarning)

def load_sample(path):
    """
    Load a .pt sample, fix up 'x' and 'edge_index', return (mol_data, pro_data, target).
    """
    sample = torch.load(path)
    mol_data, pro_data, target = sample[0], sample[1], sample[2]

    # Convert dict to Data if needed
    if isinstance(mol_data, dict):
        mol_data = Data(**mol_data)
    if isinstance(pro_data, dict):
        pro_data = Data(**pro_data)

    # Fix x
    if not hasattr(mol_data, 'x') or mol_data.x is None:
        if hasattr(mol_data, 'features'):
            mol_data.x = mol_data.features
            del mol_data.features
        else:
            raise ValueError("mol_data missing 'x' or 'features'")
    if not hasattr(pro_data, 'x') or pro_data.x is None:
        if hasattr(pro_data, 'features'):
            pro_data.x = pro_data.features
            del pro_data.features
        else:
            raise ValueError("pro_data missing 'x' or 'features'")

    mol_data.x = torch.as_tensor(mol_data.x, dtype=torch.float32)
    pro_data.x = torch.as_tensor(pro_data.x, dtype=torch.float32)

    # Fix edge_index
    def fix_edge_index(d):
        if not isinstance(d.edge_index, torch.Tensor):
            d.edge_index = torch.tensor(d.edge_index, dtype=torch.long)
        else:
            d.edge_index = d.edge_index.long()
        if d.edge_index.shape[0] != 2:
            d.edge_index = d.edge_index.t()
        d.num_nodes = d.x.size(0)

    fix_edge_index(mol_data)
    fix_edge_index(pro_data)

    return (mol_data, pro_data, target)

def batch_loader(file_list, sample_dir, batch_size):
    """
    Yields batches of (mol_data, pro_data, target) from file_list.
    """
    batch = []
    for file_name in file_list:
        path = os.path.join(sample_dir, file_name)
        sample = load_sample(path)
        batch.append(sample)
        if len(batch) == batch_size:
            yield batch
            batch = []
    if batch:
        yield batch

##############################################################################
#              4. TRAINING / EVALUATION WITH METRICS EACH EPOCH
##############################################################################

import os
import torch

def train_and_evaluate(sample_dir, num_epochs=10, test_size=0.2, lr=0.001):
    """
    Trains the GNN model, evaluates on train & test each epoch, saves metrics + checkpoints.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Running on {device}.")

    # Gather samples
    sample_files = [f for f in os.listdir(sample_dir) if f.endswith('.pt')]
    assert len(sample_files) > 0, "No .pt files found in sample_dir!"

    # Split
    train_files, test_files = train_test_split(sample_files, test_size=test_size, random_state=42)

    # Make checkpoint dir
    training_model_dir = os.path.join(os.getcwd(), 'TrainingModel1')
    os.makedirs(training_model_dir, exist_ok=True)
    print(f"Checkpoints will be saved to: {training_model_dir}")

    metrics_path = os.path.join(training_model_dir, "training_metrics.pt")

    # Load existing metrics if available (Ensures metrics continue from previous runs)
    if os.path.exists(metrics_path):
        saved_metrics = torch.load(metrics_path)
        train_metrics = saved_metrics['train_metrics']
        test_metrics = saved_metrics['test_metrics']
        print("Loaded previous training metrics!")
    else:
        train_metrics = {'epoch': [], 'mse': [], 'ci': [], 'pearson': []}
        test_metrics = {'epoch': [], 'mse': [], 'ci': [], 'pearson': []}
        print("Starting fresh metrics tracking.")

    # Infer input dims from one sample
    sample0 = load_sample(os.path.join(sample_dir, train_files[0]))
    mol_data0, pro_data0 = sample0[0], sample0[1]
    num_features_mol = mol_data0.x.size(1)
    num_features_pro = pro_data0.x.size(1)

    # Initialize model
    model = GNNNet(num_features_mol=num_features_mol,
                   num_features_pro=num_features_pro).to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    loss_fn = MSELoss()

    # Possibly resume from checkpoint
    start_epoch = 1
    existing_checkpoints = [
        f for f in os.listdir(training_model_dir)
        if f.endswith('.pt') and f.startswith('model_epoch')
    ]
    last_ckpt_path = None  # Track previous checkpoint for deletion

    if existing_checkpoints:
        latest_ckpt = max(existing_checkpoints, key=lambda x: int(x.split('_epoch')[1].split('.pt')[0]))
        ckpt_path = os.path.join(training_model_dir, latest_ckpt)
        print(f"Loading checkpoint from {ckpt_path}")
        ckpt = torch.load(ckpt_path, map_location=device)
        model.load_state_dict(ckpt['model_state_dict'])
        optimizer.load_state_dict(ckpt['optimizer_state_dict'])
        loaded_epoch = ckpt['epoch']
        start_epoch = loaded_epoch + 1
        last_ckpt_path = ckpt_path  # Store last checkpoint path for deletion
        print(f"Resuming from epoch {start_epoch}")
    else:
        print("No existing checkpoint found; starting fresh.")

    @torch.no_grad()
    def evaluate(files):
        model.eval()
        all_preds = []
        all_targets = []
        batch_eval_size = 200
        for batch_samples in batch_loader(files, sample_dir, batch_eval_size):
            mol_list, pro_list, tgt_list = [], [], []
            for (md, pd, t) in batch_samples:
                mol_list.append(md)
                pro_list.append(pd)
                tgt_list.append(t)

            mol_batch = Batch.from_data_list(mol_list).to(device)
            pro_batch = Batch.from_data_list(pro_list).to(device)
            t_tensor = torch.tensor(tgt_list, dtype=torch.float32, device=device)

            out = model(mol_batch, pro_batch).view(-1)
            all_preds.append(out)
            all_targets.append(t_tensor)

        all_preds = torch.cat(all_preds, dim=0)
        all_targets = torch.cat(all_targets, dim=0)

        mse_val = mse_torch(all_preds, all_targets)
        ci_val = ci_vectorized(all_preds, all_targets)
        pearson_val = pearson_torch(all_preds, all_targets)
        return mse_val, ci_val, pearson_val

    # Training loop
    batch_size = 30
    for epoch in tqdm(range(start_epoch, num_epochs + 1), desc="Training", unit="epoch"):
        model.train()
        running_loss = 0.0

        for batch_samples in batch_loader(train_files, sample_dir, batch_size):
            mol_list, pro_list, tgt_list = [], [], []
            for (md, pd, t) in batch_samples:
                mol_list.append(md)
                pro_list.append(pd)
                tgt_list.append(t)

            mol_batch = Batch.from_data_list(mol_list).to(device)
            pro_batch = Batch.from_data_list(pro_list).to(device)
            t_tensor = torch.tensor(tgt_list, dtype=torch.float32, device=device).view(-1)

            optimizer.zero_grad()
            out = model(mol_batch, pro_batch).view(-1)
            loss = loss_fn(out, t_tensor)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * len(batch_samples)

        avg_loss = running_loss / len(train_files)
        tqdm.write(f"[Epoch {epoch}/{num_epochs}] Training Loss: {avg_loss:.4f}")

        # Evaluate on train & test
        train_mse, train_ci, train_pearson = evaluate(train_files)
        test_mse, test_ci, test_pearson = evaluate(test_files)

        train_metrics['epoch'].append(epoch)
        train_metrics['mse'].append(train_mse)
        train_metrics['ci'].append(train_ci)
        train_metrics['pearson'].append(train_pearson)

        test_metrics['epoch'].append(epoch)
        test_metrics['mse'].append(test_mse)
        test_metrics['ci'].append(test_ci)
        test_metrics['pearson'].append(test_pearson)

        tqdm.write(f"  Train => MSE={train_mse:.4f}, CI={train_ci:.4f}, Pearson={train_pearson:.4f}")
        tqdm.write(f"  Test  => MSE={test_mse:.4f}, CI={test_ci:.4f}, Pearson={test_pearson:.4f}")

        # Save new checkpoint
        ckpt_name = f"model_epoch{epoch}.pt"
        ckpt_path = os.path.join(training_model_dir, ckpt_name)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
        }, ckpt_path)
        tqdm.write(f"Checkpoint saved at epoch {epoch}")

        # Delete the previous checkpoint after saving the new one
        if last_ckpt_path and os.path.exists(last_ckpt_path):
            os.remove(last_ckpt_path)
            tqdm.write(f"Deleted previous checkpoint: {last_ckpt_path}")

        # Update last checkpoint path
        last_ckpt_path = ckpt_path

        # Save/Update the metrics after each epoch
        torch.save({
            'train_metrics': train_metrics,
            'test_metrics': test_metrics
        }, metrics_path)

    # Final evaluations
    final_train_mse, final_train_ci, final_train_pearson = evaluate(train_files)
    final_test_mse, final_test_ci, final_test_pearson = evaluate(test_files)
    print(f"\nFinal Train => MSE={final_train_mse:.4f}, CI={final_train_ci:.4f}, Pearson={final_train_pearson:.4f}")
    print(f"Final Test  => MSE={final_test_mse:.4f}, CI={final_test_ci:.4f}, Pearson={final_test_pearson:.4f}")

    # Save final metrics
    torch.save({
        'train_metrics': train_metrics,
        'test_metrics': test_metrics
    }, metrics_path)
    print(f"Metrics saved to {metrics_path}")

    return train_metrics, test_metrics


##############################################################################
#               5. OPTIONAL: PLOT THE SAVED METRICS
##############################################################################

def plot_metrics(checkpoint_dir='TrainingModel1'):
    """
    Load training_metrics.pt from the checkpoint_dir and plot MSE, CI, Pearson over epochs.
    """
    metrics_path = os.path.join(checkpoint_dir, "training_metrics.pt")
    if not os.path.exists(metrics_path):
        print(f"No metrics file found at {metrics_path}!")
        return

    saved_data = torch.load(metrics_path)
    train_metrics = saved_data['train_metrics']
    test_metrics = saved_data['test_metrics']
    epochs = train_metrics['epoch']

    # Plot MSE
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, train_metrics['mse'], 'o-', label='Train MSE')
    plt.plot(epochs, test_metrics['mse'], 'o-', label='Test MSE')
    plt.xlabel('Epoch')
    plt.ylabel('MSE')
    plt.title('Mean Squared Error over Epochs')
    plt.legend()
    plt.savefig(os.path.join(checkpoint_dir, "MSE_plot.png"))  
    plt.show()

    # Plot CI
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, train_metrics['ci'], 'o-', label='Train CI')
    plt.plot(epochs, test_metrics['ci'], 'o-', label='Test CI')
    plt.xlabel('Epoch')
    plt.ylabel('Concordance Index')
    plt.title('CI over Epochs')
    plt.legend()
    plt.savefig(os.path.join(checkpoint_dir, "CI_plot.png"))  
    plt.show()

    # Plot Pearson
    plt.figure(figsize=(8, 6))
    plt.plot(epochs, train_metrics['pearson'], 'o-', label='Train Pearson')
    plt.plot(epochs, test_metrics['pearson'], 'o-', label='Test Pearson')
    plt.xlabel('Epoch')
    plt.ylabel('Pearson Correlation')
    plt.title('Pearson Correlation over Epochs')
    plt.legend()
    plt.savefig(os.path.join(checkpoint_dir, "PEARSON_plot.png"))  
    plt.show()

##############################################################################
#                                 MAIN
##############################################################################

if __name__ == "__main__":
    # Adjust the paths/parameters as needed
    SAMPLE_DIR = "prepared_samples"   # Directory with your .pt samples
    NUM_EPOCHS = 250
    TEST_SPLIT = 0.2
    LR = 0.001

    # 1) Train and evaluate
    train_metrics, test_metrics = train_and_evaluate(
        sample_dir=SAMPLE_DIR,
        num_epochs=NUM_EPOCHS,
        test_size=TEST_SPLIT,
        lr=LR
    )

    # 2) Plot the metrics
    plot_metrics('TrainingModel1')



Running on cuda.
Checkpoints will be saved to: /kaggle/working/TrainingModel1
Starting fresh metrics tracking.
GNNNet Loaded
No existing checkpoint found; starting fresh.


Training:   0%|          | 0/250 [00:51<?, ?epoch/s]

[Epoch 1/250] Training Loss: 1.1407
